# Using FABRIC Xilinx FPGA U280

Your compute nodes can include FPGAs. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use Xilinx FPGA devices on FABRIC.


## Setup the Experiment

#### Import FABRIC API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create a Node

The cells below help you create a slice that contains a single node with an attached FPGA. 

In [ ]:
FPGA_CHOICE='FPGA_Xilinx_U280'

# don't edit - convert from FPGA type to a resource column name
# to use in filter lambda function below
choice_to_column = {
    "FPGA_Xilinx_U280": "fpga_u280_available",
}

column_name = choice_to_column.get(FPGA_CHOICE, "Unknown")
print(f'{column_name=}')

Give the slice and the node in it meaningful names.

In [ ]:
# name the slice and the node 
slice_name=f'My Simple FPGA Slice with {FPGA_CHOICE}'
node_name='fpga-node'

print(f'Will create slice "{slice_name}" with node "{node_name}"')

Use a lambda filter to figure out which site the node will go to.

In [ ]:
# find a site with at least one available FPGA of the selected type
site = fablib.get_random_site(filter_function=lambda x: x[column_name] > 0)
print(f'Preparing to create slice "{slice_name}" with node {node_name} in site {site}')

Create the desired slice with a FPGA component. 

In [ ]:
# Create Slice. Note that by default submit() call will poll for 360 seconds every 10-20 seconds
# waiting for slice to come up. Normal expected time is around 2 minutes. 
slice = fablib.new_slice(name=slice_name)

# Add node with a 100G drive and a couple of CPU cores (default)
node = slice.add_node(name=node_name, site=site, disk=100)
node.add_component(model=FPGA_CHOICE, name='fpga1')

#Submit Slice Request
slice.submit();

## Get the Slice

Retrieve the node information and save the management IP addresses.

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.show();

## Get the Node

Retrieve the node information and save the management IP address.


In [ ]:
node = slice.get_node(node_name) 
node.show()

fpga = node.get_component('fpga1')
fpga.show();


## FPGA PCI Device

Run the command <code>lspci</code> to see your FPGA PCI device(s). This is the raw FPGA PCI device that is not yet configured for use.  You can use the FPGAs as you would any FPGAs.

View node's FPGA

In [ ]:
command = "sudo dnf install -q -y pciutils && lspci | grep 'Xilinx'"

stdout, stderr = node.execute(command)

## Cleanup Your Experiment

In [ ]:
fablib.delete_slice(slice_name)